In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import pytest
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect

# Fixtures
from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import y
from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import exog, exog_predict

In [7]:
forecaster = ForecasterDirect(
                    regressor = LinearRegression(),
                    steps     = 5,
                    lags      = 5
                )
forecaster.fit(y=y)

recommended_n_boot = 5
for k, v in forecaster.in_sample_residuals_.items():
    forecaster.in_sample_residuals_[k] = v[:recommended_n_boot]
    
results = forecaster.predict_bootstrapping(
    steps=5, n_boot=recommended_n_boot, use_in_sample_residuals=True, use_binned_residuals=False
)
results

,pred_boot_0,pred_boot_1,pred_boot_2,pred_boot_3,pred_boot_4
50,0.487216,1.024844,0.619060,0.567939,0.508131
51,0.959089,0.615396,0.493758,0.436611,0.374225
52,0.587082,0.382151,0.354616,0.350196,0.775205
53,0.384072,0.299838,0.297272,0.734615,0.515312
54,0.296045,0.285618,0.735046,0.523137,0.054519


In [8]:
recommended_n_boot = 5
in_sample_residuals_by_bin_ = forecaster.in_sample_residuals_by_bin_.copy()
for k, v in in_sample_residuals_by_bin_.items():
    in_sample_residuals_by_bin_[k] = v[:recommended_n_boot]

in_sample_residuals_by_bin_

{0: array([ 0.35398646, -0.01070607, -0.0121861 ,  0.06634067, -0.14553897]),
 1: array([ 0.01079722,  0.31667074,  0.02045466, -0.03134715, -0.18923835]),
 2: array([-0.0450625 , -0.04970559, -0.27109241, -0.2054548 , -0.13023932]),
 3: array([ 0.01474554, -0.12607334, -0.40848077,  0.1781821 ,  0.24127148]),
 4: array([-0.06597795, -0.06247223, -0.30764313,  0.11784057,  0.39257062]),
 5: array([ 0.47165077, -0.01760355, -0.39344615,  0.47603728, -0.41994487]),
 6: array([ 0.21672936,  0.00902857, -0.11833943,  0.20465353,  0.31980565]),
 7: array([-0.01111603, -0.29143333, -0.10171245, -0.05190446, -0.23149042]),
 8: array([-0.46599771, -0.13167749, -0.2400648 ,  0.38610882, -0.04943982]),
 9: array([ 0.06586613,  0.27294283,  0.13207642, -0.05288698, -0.14451512])}

In [9]:
results.to_numpy()

array([[0.48721565, 1.02484437, 0.61905973, 0.56793914, 0.5081311 ],
       [0.9590885 , 0.61539607, 0.49375752, 0.43661094, 0.37422496],
       [0.58708183, 0.38215125, 0.35461588, 0.3501964 , 0.77520526],
       [0.38407239, 0.29983844, 0.29727213, 0.73461524, 0.51531151],
       [0.29604462, 0.28561801, 0.73504556, 0.52313719, 0.05451923]])

In [11]:
forecaster.binner_intervals_

{0.0: (77.11104390221573, 95.19313158258132),
 1.0: (95.19313158258132, 107.10836783689707),
 2.0: (107.10836783689707, 122.98049619443195)}

In [ ]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.exceptions import NotFittedError
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor

from skforecast.preprocessing import RollingFeatures
from skforecast.preprocessing import TimeSeriesDifferentiator
from skforecast.recursive import ForecasterRecursive
from scipy.stats import norm
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from skforecast.model_selection import TimeSeriesFold, backtesting_forecaster

# Fixtures
from skforecast.recursive.tests.tests_forecaster_recursive.fixtures_forecaster_recursive import y
from skforecast.recursive.tests.tests_forecaster_recursive.fixtures_forecaster_recursive import exog
from skforecast.recursive.tests.tests_forecaster_recursive.fixtures_forecaster_recursive import exog_predict
from skforecast.recursive.tests.tests_forecaster_recursive.fixtures_forecaster_recursive import data  # to test results when using differentiation

In [17]:
forecaster = ForecasterDirect(
        regressor=LinearRegression(), steps=5, lags=3, binner_kwargs={'n_bins': 10}
    )
forecaster = ForecasterRecursive(
        regressor=LinearRegression(), lags=3, binner_kwargs={'n_bins': 10}
    )
n_backtest = 12
y_train = y[:-n_backtest]
cv = TimeSeriesFold(
             steps              = 5,
             initial_train_size = len(y_train),
             refit              = True
        )
metric, backtest_predictions = backtesting_forecaster(
                                    forecaster              = forecaster,
                                    y                       = y,
                                    exog                    = exog,
                                    cv                      = cv,
                                    metric                  = 'mean_squared_error',
                                    interval                = 0.9,
                                    interval_method         = 'conformal',
                                    use_in_sample_residuals = True,
                                    use_binned_residuals    = True,
                                    random_state            = 123,
                                    verbose                 = False,
                                    show_progress           = False
                                )

In [18]:
# Increase number of decimals numpy
np.set_printoptions(precision=15)
metric.to_numpy()

array([[0.064250191230055]])

In [19]:
np.set_printoptions(precision=8)
backtest_predictions.to_numpy()

array([[0.59059622, 0.2974502 , 0.88374223],
       [0.47257504, 0.22356317, 0.72158691],
       [0.53024098, 0.14339956, 0.91708239],
       [0.46163343, 0.21262156, 0.7106453 ],
       [0.50035119, 0.35604735, 0.64465502],
       [0.41975558, 0.20182965, 0.63768151],
       [0.4256614 , 0.03773832, 0.81358449],
       [0.41176005, 0.19383413, 0.62968598],
       [0.52357817, 0.43392621, 0.61323014],
       [0.509974  , 0.42032204, 0.59962597],
       [0.65354628, 0.29763244, 1.00946013],
       [0.48210726, 0.17026625, 0.79394828]])